Required packages: vfb-connect and python-catmaid (pymaid & navis)

In [1]:
!pip install vfb-connect --upgrade
!pip install python-catmaid --upgrade

### A note on using these notebooks
This is designed as an interactive tutorial. Feel free to add code cells below each example to try out variations of your own.

### How to find neurons across datasets

VirtualFlyBrain integrates images and connectomics profiles of neurons from many sources.  It classifies and records their properties using a standard, queryable classification ([The Drosophila Anatomy Ontology](https://www.ebi.ac.uk/ols/ontologies/fbbt)). This standardises the names of neuron types across sources, so you don't need to worry about differences in nomenclature uses and supports queries for neurons by their classification. 

In [2]:
# Import libs and initialise API objects
from vfb_connect.cross_server_tools import VfbConnect
import pandas as pd
vc = VfbConnect()

import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')


### Finds neurons by type (classification) across datasets

We can use the `vc.get_instances` method in combination with theNname or Symbol of a neuron type on VFB to find individual neurons from multiple sources.

Use the search tool on [VFB](http://virtualflybrain.org) to find neuron types by name or synonym:

<img src="https://user-images.githubusercontent.com/112839/109564687-535c9380-7ad9-11eb-80e9-5a5bc21cd915.png" width=30% height=30%>
<img src="https://user-images.githubusercontent.com/112839/109565128-eac1e680-7ad9-11eb-9649-1ec55a298a1b.png" width=30% height=30%>

Use either the full name or the Symbol to query for neurons:

In [3]:
DA3adPN = vc.get_instances("DA3_adPN", summary=True)
pd.DataFrame.from_records(DA3adPN)

### Find neurons by location

We can use the same method to search for neurons by location, using simple queries.  See [Semantic query guide](https://vfb-connect.readthedocs.io/en/latest/Semantic_queries.html) for more info.

In [4]:
# Find neurons by location. The following query works across multiple data sources and both sides of the brain.  
# Results may be incomplete & may include minor overlap inferred from low synapse counts

neurons_in_DA3 = vc.get_instances("'neuron' that 'overlaps' some 'antennal lobe glomerulus DA3'", summary=True)
neurons_in_DA3_tab = pd.DataFrame.from_records(neurons_in_DA3)
neurons_in_DA3_tab[0:5]


In [5]:
# Find local interneurons (intrinsic neurons) of the AL, overlapping DA3:

local_in_DA3 = vc.get_instances("'local interneuron of adult antennal lobe' that 'overlaps' some 'antennal lobe glomerulus DA3'",
                                summary=True)
local_in_DA3_tab = pd.DataFrame.from_records(local_in_DA3)
local_in_DA3_tab

In [6]:
# Find neurons by dataset/paper - on CATMAID

bates = pymaid.find_neurons(annotations='Paper: Bates and Schlegel et al 2020')
bates

In [7]:
# Inspect what datasets are available on VFB

ds = vc.neo_query_wrapper.get_datasets(summary=True)
ds_tab = pd.DataFrame.from_records(ds)
ds_tab.sort_values(by=['id'])

You can also find linked datasets directly on pub reports on VFB ![image](https://user-images.githubusercontent.com/112839/116908789-36f4e880-ac3b-11eb-9dc7-a2493917f179.png)






In [8]:
sayin_tab = pd.DataFrame.from_records(vc.get_instances_by_dataset('Sayin2019', summary=True))
sayin_tab

In [9]:
# Find connections between neurons by type across datasets.
# Note: results are limited by how detailed annotation currently is in the DB.  
    
vc.get_connected_neurons_by_type(upstream_type='GABAergic neuron', downstream_type='adult descending neuron', weight=10).sort_values(['weight'],ascending=False)